In [1]:
import ee
import datetime
import os
import itertools
import sys

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('..')

##### Define_Basic_Parameters

In [4]:
# define the num which represent the true number of verified sample-point sets
num = 5


# define the year range
year_start = [f'{i}-01-01' for i in range(1987,2020,3)]
year_end   = [f'{i}-12-31' for i in range(1989,2020,3)]

year_range = list(zip(year_start,year_end))[-num:]

year_name_all  = list([f'{span[0]}_{span[1]}' for span in zip(range(1987,2020,3),range(1989,2020,3)) ])
year_name = year_name_all[-num:]

In [5]:
year_name

['2005_2007', '2008_2010', '2011_2013', '2014_2016', '2017_2019']

In [6]:
# import verified points
Verified_Point = [ee.FeatureCollection(f"users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Verified_pt_{year}")
                              for year in year_name ]

Zone_sample    = ee.FeatureCollection("users/Jinzhu_Deakin/North_China_Plain/Verified_points_with_zone_sample/Zone_Sample")

# import north_china_plain boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

##### Prepare needed images

In [7]:
# import Landsat img
Landsat_img = [ee.Image(f"users/wang8052664/Cloud_Free_Img/Landsat_cloud_free_{year}")\
                 .clip(North_China_Plain)  for year in year_name]

# import Fourier img   
Fourier_img_all = [ee.Image(f"users/Jinzhu_Deakin/North_China_Plain/Fourier_img_harmonic_3/Fourier_img_{year}_harmonic_3")\
                     .clip(North_China_Plain)  for year in year_name_all[:6]] + \
                  [ee.Image(f"users/wang8052664/North_China_Plain/Fourier_imgs/Fourier_img_{year}_harmonic_3")\
                     .clip(North_China_Plain)  for year in year_name_all[6:]] 

Fourier_img = Fourier_img_all[-num:]

# Import the Index-mean Img
NDVI_img = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_NDVI").rename('Mean_NDVI')  
            for year in year_name ]
NDBI_img = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_NDBI").rename('Mean_NDBI')  
            for year in year_name ]
EVI_img  = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_EVI").rename('Mean_EVI')  
            for year in year_name ]

# stack Landsat_Fourier_Nomalized together
Stack_img = [ee.Image(img) for img in zip(Landsat_img,Fourier_img_all,NDVI_img,NDBI_img,EVI_img)]

##### Img value extraction and export

In [8]:
# define the gee-asset path for exporting
path = 'users/Jinzhu_Deakin/North_China_Plain/Sample_with_Landsat_Fourier_Normalized'


for year,img,sample in list(zip(year_name,Stack_img,Verified_Point))[::-1]:
    
    
    Ext_sample = img.sampleRegions(collection=sample,      properties=['Built'], scale=30, geometries=True) 
    Ext_zone   = img.sampleRegions(collection=Zone_sample, properties=['Built'], scale=30, geometries=True)
    
    # exporting
    task_sample = ee.batch.Export.table.toAsset(   collection  = Ext_sample,
                                            description = f'Verified_point_{year}_extract_Landsat_Fourier_Normalized_img',
                                            assetId     = f'{path}/Verified_point_{year}_extract_Landsat_Fourier_Normalized_img')

    
    
    task_zone = ee.batch.Export.table.toAsset(   collection  = Ext_zone,
                                            description = f'Zone_point_{year}_extract_Landsat_Fourier_Normalized_img',
                                            assetId     = f'{path}/Zone_point_{year}_extract_Landsat_Fourier_Normalized_img')
    
    task_sample.start()
    task_zone.start()
    
    # print out the process
    print(f'Zone/Sample_{year}_extract_Landsat_Fourier_Normalized_img')

Zone/Sample_2017_2019_extract_Landsat_Fourier_Normalized_img
Zone/Sample_2014_2016_extract_Landsat_Fourier_Normalized_img
Zone/Sample_2011_2013_extract_Landsat_Fourier_Normalized_img
Zone/Sample_2008_2010_extract_Landsat_Fourier_Normalized_img
Zone/Sample_2005_2007_extract_Landsat_Fourier_Normalized_img
